# Sectoral Report 
**Aim** Create short report on sectoral transition in given country and sector, answering key questions relevant to ASCOR analysts

Input: sector, country

Output: one paragraph answer to key questions

Key questions:

1. Have they set a net zero target? 
2. Have they quantified the level of their targets? 
3. (CHALLENGING) How has the legislation of this country changed in the past 50 years? 
Break into time periods, each evaluated by a different LLM and summarize

Compile into markdown with enforced format

Markdowntemplate

# ASCOR Sectoral Transition Report

## Executive Summary
**Country:** [COUNTRY NAME]  
**Sector:** [SECTOR NAME]  
---

## Key Findings

### 1. Net Zero Target Assessment

**Question:** Have they set a net zero target?

[One paragraph response analyzing whether the specified country has established net zero commitments for the given sector. Include timeline details, scope of coverage, and any conditional aspects of the target.]

**Key Details:**
- Target Date: [DATE or "Not Set"]
- Scope: [Economy-wide/Sectoral/Conditional]
- Legal Status: [Legally Binding/Policy Commitment/Aspirational]

**Confidence in Assessment:** 

### 2. Legislative Evolution Assessment

**Question:** How has the legislation of this country changed in the past 50 years?

[One paragraph response analyzing the evolution of climate and sectoral legislation over the past five decades. Focus on major policy shifts, regulatory frameworks, and legislative milestones relevant to the specified sector.]

**Legislative Timeline:**
- **1970s-1980s:** [Key developments]
- **1990s-2000s:** [Key developments]
- **2010s-Present:** [Key developments]

**Confidence in Assessment:** 


### 0. Setup

In [3]:
import os
from openai import OpenAI

import os
from openai import OpenAI
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema import BaseOutputParser
import json
from dotenv import load_dotenv

load_dotenv()


from typing_extensions import Annotated, TypedDict
from typing import Literal
from langchain_openai import ChatOpenAI
from langchain.schema.runnable import RunnablePassthrough, RunnableLambda
from typing import Dict, Any
import json

In [5]:
# Load Models
# Load models
super_basic_model = ChatOpenAI(
    base_url="https://api.studio.nebius.com/v1/",
    api_key=os.environ.get("NEBIUS_API_KEY"),
    model="meta-llama/Llama-3.2-1B-Instruct",
    temperature=0
)

standard_model = ChatOpenAI(
    base_url="https://api.studio.nebius.com/v1/",
    api_key=os.environ.get("NEBIUS_API_KEY"),
    model="meta-llama/Meta-Llama-3.1-8B-Instruct",
    temperature=0
)

larger_context_model = ChatOpenAI(
    base_url="https://api.studio.nebius.com/v1/",
    api_key=os.environ.get("NEBIUS_API_KEY"),
    model="meta-llama/Meta-Llama-3.1-70B-Instruct",
    temperature=0
)

### 1. Enforce Output schema


In [6]:
from typing_extensions import Annotated, TypedDict
from typing import Literal

class NetZeroTargetDetails(TypedDict):
    """Structured details for net zero target assessment"""
    target_date: Annotated[str, ..., "Target achievement date or 'Not Set'"]
    scope: Annotated[Literal["Economy-wide", "Sectoral", "Conditional"], ..., "Target scope classification"]
    legal_status: Annotated[Literal["Legally Binding", "Policy Commitment", "Aspirational"], ..., "Legal status of the target"]

class NetZeroAssessment(TypedDict):
    """Complete net zero target assessment from dedicated LLM"""
    net_zero_response: Annotated[str, ..., "One paragraph analyzing net zero commitments, including timeline, scope, and conditional aspects"]
    net_zero_details: Annotated[NetZeroTargetDetails, ..., "Structured key details about the net zero target"]
    net_zero_confidence: Annotated[Literal["High", "Medium", "Low"], ..., "Confidence level in the net zero assessment"]

class PeriodAnalysis1970s(TypedDict):
    """1970s-1980s legislative analysis result"""
    period_1970s_1980s: Annotated[str, ..., "Key legislative developments in 1970s-1980s period"]

class PeriodAnalysis1990s(TypedDict):
    """1990s-2000s legislative analysis result"""
    period_1990s_2000s: Annotated[str, ..., "Key legislative developments in 1990s-2000s period"]

class PeriodAnalysis2010s(TypedDict):
    """2010s-Present legislative analysis result"""
    period_2010s_present: Annotated[str, ..., "Key legislative developments in 2010s-Present period"]

class LegislativeTimeline(TypedDict):
    """Timeline breakdown of legislative changes from dedicated LLM"""
    period_1970s_1980s: Annotated[str, ..., "Key legislative developments in 1970s-1980s period"]
    period_1990s_2000s: Annotated[str, ..., "Key legislative developments in 1990s-2000s period"]  
    period_2010s_present: Annotated[str, ..., "Key legislative developments in 2010s-Present period"]

class LegislativeSummary(TypedDict):
    """Legislative evolution summary from synthesis LLM"""
    legislation_response: Annotated[str, ..., "One paragraph analyzing 50 years of legislative evolution, focusing on policy shifts and regulatory frameworks"]
    legislation_confidence: Annotated[Literal["High", "Medium", "Low"], ..., "Confidence level in the legislative assessment"]

class SimpleASCORReport(TypedDict):
    """Final assembled report structure"""
    
    # Executive Summary fields
    country: Annotated[str, ..., "Name of the country being analyzed"]
    sector: Annotated[str, ..., "Name of the sector being analyzed"]
    
    # Net Zero Target Assessment (from specialized LLM)
    net_zero_response: Annotated[str, ..., "One paragraph analyzing net zero commitments"]
    net_zero_details: Annotated[NetZeroTargetDetails, ..., "Structured key details about the net zero target"]
    net_zero_confidence: Annotated[Literal["High", "Medium", "Low"], ..., "Confidence level in the net zero assessment"]
    
    # Legislative Evolution Assessment (assembled from multiple LLMs)
    legislation_response: Annotated[str, ..., "One paragraph analyzing 50 years of legislative evolution"]
    legislative_timeline: Annotated[LegislativeTimeline, ..., "Structured timeline of legislative changes by period"]
    legislation_confidence: Annotated[Literal["High", "Medium", "Low"], ..., "Confidence level in the legislative assessment"]

### 2. Get Context

### 2. Prompts

In [7]:
# Separation of Concerns: Individual LLM Functions
def analyze_net_zero(inputs: Dict[str, Any]) -> NetZeroAssessment:
    """Dedicated LLM for net zero analysis - uses larger context model for complexity"""
    llm = larger_context_model.with_structured_output(NetZeroAssessment)
    
    prompt = f"""Analyze net zero targets for {inputs['country']} in {inputs['sector']} sector.
    
    Context: {inputs.get('context', 'No specific context provided')}
    
    Provide:
    1. One paragraph analysis of net zero commitments
    2. Structured target details (date, scope, legal status)
    3. Confidence assessment
    
    For any claims you make, you **MUST** include the page number and document citation in the format (page X, doc Y).

    """
    
    return llm.invoke(prompt)

def analyze_period_1970s_1980s(inputs: Dict[str, Any]) -> PeriodAnalysis1970s:
    """Dedicated LLM for 1970s-1980s legislative analysis"""
    prompt = f"""Analyze 1970s-1980s legislation for {inputs['country']} {inputs['sector']} sector.
    Focus on major policy shifts, regulatory frameworks shifts, and legislative milestones relevant to climate policy in the sector.
    For any claims you make, you **MUST** include the page number and document citation in the format (page X, doc Y).
    Context: {inputs.get('context', '')}
    """
    
    result = super_basic_model.invoke(prompt)
    return PeriodAnalysis1970s(period_1970s_1980s=result.content)

def analyze_period_1990s_2000s(inputs: Dict[str, Any]) -> PeriodAnalysis1990s:
    """Dedicated LLM for 1990s-2000s legislative analysis"""
    prompt = f"""Analyze 1990s-2000s legislation for {inputs['country']} {inputs['sector']} sector.
    Focus on major policy shifts, regulatory frameworks shifts, and legislative milestones relevant to climate policy in the sector.
    For any claims you make, you **MUST** include the page number and document citation in the format (page X, doc Y).
    Context: {inputs.get('context', '')}
    """
    
    result = standard_model.invoke(prompt)
    return PeriodAnalysis1990s(period_1990s_2000s=result.content)

def analyze_period_2010s_present(inputs: Dict[str, Any]) -> PeriodAnalysis2010s:
    """Dedicated LLM for 2010s-Present legislative analysis - uses larger model for recent complex policies"""
    prompt = f"""Analyze 2010s-Present legislation for {inputs['country']} {inputs['sector']} sector.
    Focus on major policy shifts, regulatory frameworks shifts, and legislative milestones relevant to climate policy in the sector.
    For any claims you make, you **MUST** include the page number and document citation in the format (page X, doc Y).
    Context: {inputs.get('context', '')}
    """
    
    result = larger_context_model.invoke(prompt)
    return PeriodAnalysis2010s(period_2010s_present=result.content)

def synthesize_legislative_timeline(timeline_parts: Dict[str, Any]) -> LegislativeTimeline:
    """Combine timeline periods into structured output"""
    return LegislativeTimeline(
        period_1970s_1980s=timeline_parts["period_1970s_1980s"],
        period_1990s_2000s=timeline_parts["period_1990s_2000s"],
        period_2010s_present=timeline_parts["period_2010s_present"]
    )

def synthesize_legislative_summary(inputs: Dict[str, Any]) -> LegislativeSummary:
    """Dedicated LLM for synthesizing timeline into coherent summary - uses standard model for synthesis"""
    llm = standard_model.with_structured_output(LegislativeSummary)
    
    timeline = inputs["legislative_timeline"]
    prompt = f"""Synthesize 50 years of legislative evolution for {inputs['country']} {inputs['sector']}.
    
    Timeline data:
    - 1970s-1980s: {timeline['period_1970s_1980s']}
    - 1990s-2000s: {timeline['period_1990s_2000s']}
    - 2010s-Present: {timeline['period_2010s_present']}
    
    Create one flowing paragraph showing policy evolution and assess confidence.
    For any claims you make, you **MUST** include the page number and document citation in the format (page X, doc Y).
    """
    
    return llm.invoke(prompt)

def assemble_final_report(inputs: Dict[str, Any]) -> SimpleASCORReport:
    """Assemble all components into final report structure"""
    return SimpleASCORReport(
        country=inputs["country"],
        sector=inputs["sector"],
        net_zero_response=inputs["net_zero_assessment"]["net_zero_response"],
        net_zero_details=inputs["net_zero_assessment"]["net_zero_details"],
        net_zero_confidence=inputs["net_zero_assessment"]["net_zero_confidence"],
        legislation_response=inputs["legislative_summary"]["legislation_response"],
        legislative_timeline=inputs["legislative_timeline"],
        legislation_confidence=inputs["legislative_summary"]["legislation_confidence"]
    )

# Main Workflow Chain
def create_ascor_workflow():
    """Create the complete ASCOR report generation workflow"""
    
    # Parallel legislative analysis for each time period
    legislative_chain = (
        RunnablePassthrough()
        | {
            "period_1970s_1980s": RunnableLambda(analyze_period_1970s_1980s),
            "period_1990s_2000s": RunnableLambda(analyze_period_1990s_2000s), 
            "period_2010s_present": RunnableLambda(analyze_period_2010s_present)
        }
        | RunnableLambda(lambda x: {k: v[k] for k, v in x.items()})  # Flatten results
        | RunnableLambda(synthesize_legislative_timeline)
    )
    
    # Complete workflow
    workflow = (
        RunnablePassthrough()
        | {
            "country": lambda x: x["country"],
            "sector": lambda x: x["sector"], 
            "net_zero_assessment": RunnableLambda(analyze_net_zero),
            "legislative_timeline": legislative_chain
        }
        | {
            "country": lambda x: x["country"],
            "sector": lambda x: x["sector"],
            "net_zero_assessment": lambda x: x["net_zero_assessment"],
            "legislative_timeline": lambda x: x["legislative_timeline"],
            "legislative_summary": RunnableLambda(synthesize_legislative_summary)
        }
        | RunnableLambda(assemble_final_report)
    )
    
    return workflow

# Usage
def generate_ascor_report(country: str, sector: str, context: str = "") -> SimpleASCORReport:
    """Generate ASCOR report for given country and sector"""
    workflow = create_ascor_workflow()
    
    inputs = {
        "country": country,
        "sector": sector,
        "context": context
    }
    
    return workflow.invoke(inputs)

In [ ]:
def format_report_as_markdown(report: SimpleASCORReport) -> str:
    """Convert the structured report to formatted markdown"""
    
    markdown = f"""# ASCOR Sectoral Transition Report

## Executive Summary
**Country:** {report['country']}  
**Sector:** {report['sector']}  
---

## Key Findings

### 1. Net Zero Target Assessment

**Question:** Have they set a net zero target?

{report['net_zero_response']}

**Key Details:**
- Target Date: {report['net_zero_details']['target_date']}
- Scope: {report['net_zero_details']['scope']}
- Legal Status: {report['net_zero_details']['legal_status']}

**Confidence in Assessment:** {report['net_zero_confidence']}

### 2. Legislative Evolution Assessment

**Question:** How has the legislation of this country changed in the past 50 years?

{report['legislation_response']}

**Legislative Timeline:**
- **1970s-1980s:** {report['legislative_timeline']['period_1970s_1980s']}
- **1990s-2000s:** {report['legislative_timeline']['period_1990s_2000s']}
- **2010s-Present:** {report['legislative_timeline']['period_2010s_present']}

**Confidence in Assessment:** {report['legislation_confidence']}
"""
    
    return markdown

In [8]:
test_context = {
    "documents": [
        {
            "title": "Germany Energy Transition Policy Review 2023",
            "pages": 45,
            "content": {
                "net_zero_commitments": "Germany has committed to achieving net zero emissions by 2045, with interim targets of 65% reduction by 2030 (page 12, doc 1). The Energiewende program specifically targets 80% renewable electricity by 2030 (page 15, doc 1).",
                "legislation_1970s_1980s": "The Federal Emission Control Act (Bundes-Immissionsschutzgesetz) was enacted in 1974, establishing the foundation for environmental regulation (page 8, doc 1). Energy conservation laws were introduced following the 1979 oil crisis (page 9, doc 1).",
                "legislation_1990s_2000s": "The Renewable Energy Act (EEG) was passed in 2000, creating feed-in tariffs for renewable energy (page 18, doc 1). EU emissions trading system integration occurred in 2005 (page 20, doc 1).",
                "legislation_2010s_present": "The Climate Action Law (Klimaschutzgesetz) was enacted in 2019, setting legally binding sector-specific emission targets (page 25, doc 1). Coal phase-out legislation passed in 2020 mandating closure by 2038 (page 28, doc 1)."
            }
        },
        {
            "title": "European Energy Sector Analysis 2022",
            "pages": 67,
            "content": {
                "regulatory_framework": "EU Green Deal establishes framework for member state climate policies (page 23, doc 2). Fit for 55 package updates emission reduction targets to 55% by 2030 (page 31, doc 2)."
            }
        }
    ],
    "sector_specifics": {
        "energy": {
            "key_technologies": ["wind", "solar", "hydrogen", "energy storage"],
            "major_players": ["RWE", "E.ON", "EnBW", "Vattenfall"],
            "challenges": ["grid stability", "storage capacity", "industrial transition"]
        }
    }
}


In [11]:
test_context_json = json.dumps(test_context, indent=2)


def test_ascor_workflow():
    """Test the workflow with fake data"""
    
    # Test inputs
    test_country = "Germany"
    test_sector = "Energy"
    
    # Run the workflow
    try:
        report = generate_ascor_report(
            country=test_country,
            sector=test_sector, 
            context=test_context_json
        )
        
        print("✅ Workflow completed successfully!")
        print(f"Report for {report['country']} - {report['sector']} sector")
        print(f"Net Zero Target Date: {report['net_zero_details']['target_date']}")
        print(f"Confidence: {report['net_zero_confidence']}")
        
        return report
        
    except Exception as e:
        print(f"❌ Workflow failed: {e}")
        return None
    
test_result = test_ascor_workflow()

✅ Workflow completed successfully!
Report for Germany - Energy sector
Net Zero Target Date: 2045
Confidence: Medium


In [17]:
markdown_report = format_report_as_markdown(test_result)
print(markdown_report)

# ASCOR Sectoral Transition Report

## Executive Summary
**Country:** Germany  
**Sector:** Energy  
---

## Key Findings

### 1. Net Zero Target Assessment

**Question:** Have they set a net zero target?

Germany has made significant commitments to achieving net zero emissions, with a target of 2045 and interim reductions of 65% by 2030 (page 12, doc 1). The Energiewende program specifically targets 80% renewable electricity by 2030 (page 15, doc 1), demonstrating a strong focus on the energy sector. These targets are supported by legislation such as the Climate Action Law (Klimaschutzgesetz) enacted in 2019, which sets legally binding sector-specific emission targets (page 25, doc 1). However, challenges remain, including grid stability, storage capacity, and industrial transition (sector_specifics, energy, challenges).

**Key Details:**
- Target Date: 2045
- Scope: Economy-wide
- Legal Status: Legally Binding

**Confidence in Assessment:** Medium

### 2. Legislative Evolution Assess